# Import module

In [5]:
#from transformers import AutoTokenizer
import tensorflow as tf
import numpy as np
import pickle

In [6]:
# Chúng ta có thể sử dụng phobert để tokenize chữ tiếng Việt với bộ từ điển lớn hơn
#tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')
# Tất nhiên khi sử dụng tokenizer trên bộ từ điển của mình thì sẽ có thể gặp trường hợp bị thiếu từ và có thể sẽ bị lỗi khi hiện thực (vì chữ trong input không có trong tokenizer của mình)
# Nên để sử dụng được cho nhiều trường hợp thì ta nên dùng phobert đã train sẵn và sử dụng với bộ từ điển lớn và chính xác hơn
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~ ') # không để dấu gạch dưới _

# Load file đã Tiền xử lý

In [7]:
# Đọc dữ liệu từ file train.txt được tạo sau khi đã tiền xử lý dữ liệu
sequences = []
with open('train.txt', 'r', encoding='utf-16') as f:
    for line in f.readlines():
        sequences.append(line.strip()) # do khi đọc file sẽ có dấu \n và hàm strip sẽ bỏ nó đi

In [8]:
# Hàm fit_on_texts dùng để chuyển các từ thành số như một bảng ánh xạ (vì model chỉ tính toán trên những con số)
tokenizer.fit_on_texts(sequences)
# Lưu tokenizer lại để sử dụng cho việc hiện thực hóa model
with open('tokenizer.pkl', 'wb') as f:
  pickle.dump(tokenizer, f)

In [9]:
# sau khi fit_on_texts, ta tiến hành chuyển nó thành số dựa trên bảng đã tạo
sequences_digit = tokenizer.texts_to_sequences(sequences)
sequences_digit = np.array(sequences_digit)

In [10]:
vocab_size = len(tokenizer.word_index) + 1
# Lấy đầu vào và đầu ra phù hợp với model
X, y = sequences_digit[:, :-1], sequences_digit[:,-1]
X.shape, y.shape

((305461, 50), (305461,))

# Xây dựng model

In [11]:
from tensorflow.keras import layers

In [12]:
# Khởi tạo model RNN
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, 64, input_length=X.shape[1]),
    layers.BatchNormalization(),
    layers.Bidirectional(layers.LSTM(256, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(256, return_sequences=False)),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(vocab_size, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 64)            931520    
                                                                 
 batch_normalization (BatchN  (None, 50, 64)           256       
 ormalization)                                                   
                                                                 
 bidirectional (Bidirectiona  (None, 50, 512)          657408    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 512)              1574912   
 nal)                                                            
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                        

In [17]:
# Tiến hành huấn luyện model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=20, batch_size=256)

Epoch 1/20
1194/1194 [==============================] - 171s 107ms/step - loss: 6.0153 - accuracy: 0.1061
Epoch 2/20
1194/1194 [==============================] - 98s 82ms/step - loss: 5.4419 - accuracy: 0.1329
Epoch 3/20
1194/1194 [==============================] - 96s 80ms/step - loss: 5.0129 - accuracy: 0.1571
Epoch 4/20
1194/1194 [==============================] - 94s 79ms/step - loss: 4.6339 - accuracy: 0.1817
Epoch 5/20
1194/1194 [==============================] - 94s 78ms/step - loss: 4.2857 - accuracy: 0.2098
Epoch 6/20
1194/1194 [==============================] - 93s 78ms/step - loss: 3.9686 - accuracy: 0.2399
Epoch 7/20
1194/1194 [==============================] - 94s 79ms/step - loss: 3.6722 - accuracy: 0.2722
Epoch 8/20
1194/1194 [==============================] - 92s 77ms/step - loss: 3.4053 - accuracy: 0.3048
Epoch 9/20
1194/1194 [==============================] - 92s 77ms/step - loss: 3.1601 - accuracy: 0.3365
Epoch 10/20
1194/1194 [==============================] - 92s 7

In [20]:
# Huấn luyện thêm có thể chạy bao nhiêu lần tùy thích (để đạt được accuracy: 0.7456 mình đã chạy khoảng 70 epoch nhé!)
model.fit(X, y, epochs=10, batch_size=256)
model.save('weight_lstm_model.h5')

Epoch 1/10
1194/1194 [==============================] - 90s 76ms/step - loss: 1.1241 - accuracy: 0.6901
Epoch 2/10
1194/1194 [==============================] - 92s 77ms/step - loss: 1.0902 - accuracy: 0.6969
Epoch 3/10
1194/1194 [==============================] - 92s 77ms/step - loss: 1.0546 - accuracy: 0.7069
Epoch 4/10
1194/1194 [==============================] - 92s 77ms/step - loss: 1.0282 - accuracy: 0.7126
Epoch 5/10
1194/1194 [==============================] - 92s 77ms/step - loss: 1.0046 - accuracy: 0.7179
Epoch 6/10
1194/1194 [==============================] - 92s 77ms/step - loss: 0.9793 - accuracy: 0.7233
Epoch 7/10
1194/1194 [==============================] - 92s 77ms/step - loss: 0.9556 - accuracy: 0.7296
Epoch 8/10
1194/1194 [==============================] - 92s 77ms/step - loss: 0.9308 - accuracy: 0.7354
Epoch 9/10
1194/1194 [==============================] - 92s 77ms/step - loss: 0.9068 - accuracy: 0.7415
Epoch 10/10
1194/1194 [==============================] - 92s 77m